# 실행 코드

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm 
import pandas as pd

In [2]:
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("20220101", "20220531")

In [3]:
data = []
for date in dates:
    req = requests.get('https://news.daum.net/breakingnews/politics?page=1000&regDate={0}'.format(date)) #임의로 1000페이지 로드
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    # 날짜별 총 페이지수 수집
    total_page = soup.find_all('em',class_="num_page")[0].get_text()[7:-17] #'현재페이지'라는 문자랑 공백 삭제 
    
    for page in tqdm(range(1,int(total_page)+1)):
        req = requests.get('https://news.daum.net/breakingnews/politics?page={0}&regDate={1}'.format(page,date))
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        
        #뉴스 기사 링크, 언론사
        links = soup.find_all('strong',class_="tit_thumb")[:-4] #배너에 뉴스 4개가 있어서 :-4해줌
        for link in links:
            url = link.find_all("a")[0].get('href')
            name = link.find_all('span',class_='info_news')[0].get_text()[:-8] #span에 시간이 같이 포함되서 :-8해줌.
            data.append((url,name,date))

100%|████████████████████████████████████████████████████████████████████████████████| 397/397 [01:36<00:00,  4.10it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [01:29<00:00,  4.07it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 323/323 [01:19<00:00,  4.05it/s]


In [4]:
df = pd.DataFrame(
    data, columns=['url', 'name', 'date'])
df

,url,name,date
0,https://v.daum.net/v/20220101233414852,YTN,20220101
1,https://v.daum.net/v/20220101233124838,조선비즈,20220101
2,https://v.daum.net/v/20220101232618814,연합뉴스,20220101
3,https://v.daum.net/v/20220101232154794,조선비즈,20220101
4,https://v.daum.net/v/20220101231742779,뉴스1,20220101
...,...,...,...
516581,https://v.daum.net/v/20220531000049111,데일리안,20220531
516582,https://v.daum.net/v/20220531000048110,더팩트,20220531
516583,https://v.daum.net/v/20220531000045109,더팩트,20220531
516584,https://v.daum.net/v/20220531000037103,조선일보,20220531


In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

url     0
name    0
date    0
dtype: int64

In [7]:
df.to_csv("df.csv",index=False, encoding="utf-8-sig")

In [8]:
data_load = pd.read_csv("df.csv")

In [9]:
df['name'].unique()

array(['YTN', '조선비즈', '연합뉴스', '뉴스1', '뉴시스', '매일경제', '서울경제', '오마이뉴스',
       '파이낸셜뉴스', '아시아경제', '디지털타임스', 'KBS', '머니투데이', '한국경제', '머니S', '쿠키뉴스',
       '조선일보', '헤럴드경제', 'SBS', '중앙일보', 'MBC', '노컷뉴스', 'MBN', '세계일보',
       'JTBC', '한국일보', '채널A', '경향신문', '국민일보', '연합뉴스TV', '서울신문', '이데일리',
       '더팩트', '한겨레', '아이뉴스24', '데일리안', 'SBS Biz', '동아일보', '시사저널', '문화일보',
       '프레시안', '전자신문', '신동아', '주간동아', '시사IN', '코리아헤럴드', '코리아중앙데일리',
       '미디어오늘', '스포츠동아', '한겨레21', 'KTV', '정책브리핑', '한국경제TV', '베스트일레븐',
       '투어코리아', '주간경향', '이코노미조선', '지디넷코리아', '스포츠조선', '전주MBC', '대전일보',
       '대구MBC', '바스켓코리아', '강원도민일보', 'EBS', 'JIBS', '뉴스타파', '동아사이언스',
       'CJB청주방송', '국제신문', '여성동아', 'MK스포츠', 'kbc광주방송', '스포츠경향', '중앙SUNDAY',
       '코스모폴리탄', '스포츠서울', '일간스포츠', '스포츠한국', '우먼센스', '엘르', '베이비뉴스', '에스티엔',
       '스포티비뉴스'], dtype=object)

In [10]:
len(df['name'].unique())

84

In [20]:
df['name'].value_counts().head(10)

연합뉴스     175535
뉴시스      143725
뉴스1      139647
이데일리      46444
YTN       34819
헤럴드경제     27543
KBS       25371
머니투데이     22856
서울경제      21807
머니S       21807
Name: name, dtype: int64

# 테스트 코드

In [10]:
# req = requests.get('https://news.daum.net/breakingnews/politics?page=1000&regDate=20210103')
# req

In [11]:
# html = req.text
# soup = BeautifulSoup(html, 'html.parser')

In [12]:
# # 날짜별 총 페이지수 수집
# total_page = soup.find_all('em',class_="num_page")[0].get_text()[7:-17] #'현재페이지'라는 문자랑 공백 삭제 

In [13]:
# #뉴스 기사 링크, 언론사
# links = soup.find_all('strong',class_="tit_thumb")[:-4] #배너에 뉴스 4개가 있어서 :-4해줌
# for i in links:
#     print(i.find_all("a")[0].get('href')) 
#     print(i.find_all('span',class_='info_news')[0].get_text()[:-8]) #span에 시간이 같이 포함되서 :-8해줌.

In [14]:
# req = requests.get('https://news.v.daum.net/v/20210102002932918')
# req

In [15]:
# html = req.text
# soup = BeautifulSoup(html, 'html.parser')

In [16]:
# #댓글 개수
# soup.find('span', class_="alex-count-area").get_text()

In [22]:
req2 = requests.get('https://gall.dcinside.com/board/view/?id=football_new7&no=4453885')
req2

<Response [200]>

In [26]:
req2 = requests.get('https://www.fmkorea.com/4760644617')
html = req2.text

In [27]:
soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="noarchive" name="robots"/>
<meta content="sketchbook5_elkha" name="module_skin"/>
<meta content="no" http-equiv="imagetoolbar"/> <title>한동숙 요크시티 스킨 - FM2022 자유게시판 - 에펨코리아</title>
<script type="text/javascript"> </script>
<link href="//image.fmkorea.com/touchicon/logo180.png" rel="apple-touch-icon"/>
<link href="//image.fmkorea.com/touchicon/logo76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="//image.fmkorea.com/touchicon/logo120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="//image.fmkorea.com/touchicon/logo152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="//static.fmkorea.com/common/css/default.css?c=1646284189" media="all" rel="stylesheet" type="text/css"/>
<link href="//static.fmkorea.com/common/css/button.css?c=1652855842" media="all" rel="stylesheet" type="text/css"/>
<link href="//stat

In [29]:
BASE_URL = "https://gall.dcinside.com/board/lists"

# 파라미터 설정
params = {
            'id': 'monsterhunter',}

# 헤더 설정
headers = {
    'User-Agent' :'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36' }
resp = requests.get(BASE_URL, params=params, headers=headers)

In [30]:
resp

<Response [200]>

In [31]:
soup = BeautifulSoup(resp.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="darkmode" lang="ko">
<head>
<meta charset="utf-8"/>
<title>몬스터헌터 갤러리 - 커뮤니티 포털 디시인사이드</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="kr" name="content-language"/>
<meta content="BIrCvkIFsSjOnGLZzGAZC6sKRmwLlkH2NYeLbrLMGCc" name="google-site-verification"/>
<link href="https://gall.dcinside.com/board/lists/?id=monsterhunter" rel="canonical"/> <link href="https://m.dcinside.com/board/monsterhunter" media="only screen and (max-width: 640px)" rel="alternate"/>
<meta content="index,follow" name="robots"/>
<meta content="몬스터헌터 갤러리 - 커뮤니티 포털 디시인사이드" name="title"/>
<meta content="게임 카테고리로 분류된 몬스터헌터 갤러리입니다. 몬스터헌터 갤러리에 다양한 이야기를 남겨주세요." name="description"/>
<meta content="게임, 몬스터헌터, monsterhunter, 온라인게임,헌팅 액션,캡콤,플레이스테이션,Monster Hunter, game, action game, PS4, XBOX, RPG, dc, 디시, gall, gallery, 갤러리, 커뮤니티" name="keywords"/>
<meta content="summary" name="twitter:card"/>
<meta content="https://g